In [ ]:
import wandb
import os
import pandas as pd
import sys
import json

import seaborn as sns
import numpy as np
sns.set(style="whitegrid", palette="muted")

sys.path.insert(1, "/home/eecs/wooders/experiments/stl/offline")

In [ ]:
run = wandb.init()
results_dir = run.use_artifact('ucb-ralf/stl/results:v11', type='dataset').download()
orig_results_dir = run.use_artifact('ucb-ralf/stl/results:v4', type='dataset').download()
yahoo_train_dir = run.use_artifact('ucb-ralf/stl/yahoo_train_data:v0', type='dataset').download()
yahoo_eval_dir = run.use_artifact('ucb-ralf/stl/yahoo_eval_data:v0', type='dataset').download()
oracle_dir = run.use_artifact('ucb-ralf/stl/oracle:v0', type='dataset').download()

# Check Train / Eval Data

In [ ]:
key = 3

In [ ]:
df_train = pd.read_csv(f"{yahoo_train_dir}/{key}.csv")
plt.plot(np.arange(len(df_train)), df_train["value"])

In [ ]:
df_eval = pd.read_csv(f"{yahoo_eval_dir}/{key}.csv")
plt.plot(np.arange(len(df_eval)), df_eval["value"])

In [ ]:
df_all = pd.concat([df_train["value"], df_eval["value"]], axis = 0)

In [ ]:
plt.plot(np.arange(len(df_train) + len(df_eval)), df_all)

# Cost Evaluation 

In [ ]:
# use simon's original results 
orig_results_dir

In [ ]:
from sktime.performance_metrics.forecasting import mean_squared_scaled_error
def get_loss_per_key(path, oracle_filename):

    oracle_residual = pd.read_csv(oracle_filename)[
        "pred_residual"
    ]

    df = pd.read_csv(path)
    residual = df["pred_residual"]
    mask = ~np.isnan(residual)
    loss = mean_squared_scaled_error(
        y_true=oracle_residual[mask], y_pred=residual[mask], y_train=df["value"]
    )
    loss = {
        "loss": loss,
        "n_fits": df["model_version"].dropna().nunique(),
    }
    return loss

Get baseline results (single key)

In [ ]:
replica = 1
slides = [6, 12, 18, 24, 48, 96, 168, 192, 336, 672]
baseline_results = {}
for key in range(1, 101, 1):
    
    baseline_results[key] = []
    
    for slide in slides: 
        oracle_filename = f"{orig_results_dir}/plan_eval/oracle_key_A4Benchmark-TS{key}.csv"
        baseline_filename = f"{orig_results_dir}/plan_eval/slide_{slide}_key_A4Benchmark-TS{key}.csv"

        losses = get_loss_per_key(baseline_filename, oracle_filename)
        losses["slide_size"] = slide
        baseline_results[key].append(losses)
    

In [ ]:
baseline_results

## Compare to LP Results 

In [ ]:
experiments = [("max_fits_1100", 96), ("max_fits_2100", 48), ("max_fits_4200", 24), ("max_fits_8400", 12)]

graph_results = {"baseline": [], "optimized": [], "cost": []}

replica = 1
for plan, slide_size in experiments:
    print(plan, slide_size)
    
    baseline_total_cost = 0
    baseline_total_loss = 0
    for key in baseline_results.keys(): 
        for loss in baseline_results[key]:
            if loss['slide_size'] == slide_size:
                baseline_total_cost += loss['n_fits']
                baseline_total_loss += loss['loss']
    print("baseline", baseline_total_cost, baseline_total_loss)
    
    for key in range(1, 101, 1):
        oracle_filename = f"{orig_results_dir}/plan_eval/oracle_key_A4Benchmark-TS{key}.csv"
        filename = f"{artifact_dir}/lp_plan_eval/{plan}/{key}.csv"
        lp_results[key] = get_loss_per_key(filename, oracle_filename)
        
    lp_total_cost = 0
    lp_total_loss = 0
    for key in lp_results.keys(): 
        lp_total_cost += lp_results[key]['n_fits']
        lp_total_loss += lp_results[key]['loss']
    print("lp", lp_total_cost, lp_total_loss)
    
    assert lp_total_cost <= baseline_total_cost
    
    graph_results["baseline"].append(baseline_total_loss)
    graph_results["optimized"].append(lp_total_loss)
    graph_results["cost"].append(baseline_total_cost)


## Plot Cost (num fits) + Loss

In [ ]:
import matplotlib.pyplot as plt
import seaborn

x = 'Factor'

df = pd.DataFrame({
     x: graph_results["cost"], 
    'baseline': graph_results["baseline"], 
    "optimized": graph_results["optimized"],
})
fig, ax1 = plt.subplots(figsize=(10, 5))
tidy = df.melt(id_vars=x).rename(columns=str.title)
seaborn.barplot(x=x, y='Value', hue='Variable', data=tidy, ax=ax1)
seaborn.despine(fig)

ax1.set(xlabel="Cost Budget", ylabel=f'MASE Loss', title='Residual Estimate Loss for Time-Series Decomposition')
#ax1.legend_.remove()
#plt.legend(loc='lower center')

# Plot different numbers of replicas

In [ ]:
# use new results dir
results_dir

In [ ]:
experiments = [("max_fits_1100", 96), ("max_fits_2100", 48), ("max_fits_4200", 24), ("max_fits_8400", 12)]
replicas = [1, 2]
slides = [1, 6, 12, 18, 24, 48, 96, 168, 192, 336, 672]
graph_results = {"baseline": [], "optimized": [], "cost": []}
prio = "lifo"
baseline_replica_results = {}

for slide in slides: 
    baseline_replica_results[slide] = []
    for replica in replicas: 
        baseline_plan = f"plan_baseline_{slide}_{prio}"
        
        total_loss = 0
        for key in range(1, 101, 1):
            oracle_filename = f"{oracle_dir}/{key}.csv"
            
            lp_filename = f"{results_dir}/replica_{replica}/{baseline_plan}/{key}.csv"
            
            baseline_filename = f"{results_dir}/replica_{replica}/{baseline_plan}/{key}.csv"
            results = get_loss_per_key(baseline_filename, oracle_filename)
            #print(results)
            total_loss += results["loss"]
            
        baseline_replica_results[slide].append(total_loss)
    
baseline_replica_results

In [ ]:
import matplotlib.pyplot as plt
import seaborn
print(replicas)
x = 'Factor'

df = pd.DataFrame({
    x: replicas, 
    **baseline_replica_results,
})
fig, ax1 = plt.subplots(figsize=(10, 5))
tidy = df.melt(id_vars=x).rename(columns=str.title)
seaborn.barplot(x=x, y='Value', hue='Variable', data=tidy, ax=ax1)
seaborn.despine(fig)

ax1.set(xlabel="Num Replicas", ylabel=f'MASE Loss', title='Residual Estimate Loss for Time-Series Decomposition')

In [ ]:
experiments = [("max_fits_1100", 96), ("max_fits_2100", 48), ("max_fits_4200", 24), ("max_fits_8400", 12)]
replicas = [1, 2]
slides = [1, 6, 12, 18, 24, 48, 96, 168, 192, 336, 672]
graph_results = {"baseline": [], "optimized": [], "cost": []}

lp_replica_results = {}

for plan, slide in experiments: 
    lp_replica_results[plan] = []
    
    for replica in replicas:
        total_loss = 0
        for key in range(1, 101, 1):
            oracle_filename = f"{oracle_dir}/{key}.csv"
            lp_filename = f"{results_dir}/replica_{replica}/{plan}/{key}.csv"
            results = get_loss_per_key(lp_filename, oracle_filename)
            total_loss += results["loss"]
            
        lp_replica_results[plan].append(total_loss)
        print(plan, total_loss)
    
lp_replica_results

In [ ]:
static_results = {1: [213.2200178532724, 150.7256273025718, 133.3349003094353],
 6: [218.75801939773078, 156.0948132080653, 135.81574629721845],
 12: [220.90611494937247, 159.05690147515068, 138.24448802117672],
 18: [229.63341620779627, 163.1813146667572, 140.3471142793597],
 24: [233.91725185369373, 164.71255155633278, 144.22561887879107],
 48: [268.3977607679711, 184.14616983478135, 158.3310894771346],
 96: [348.50466291276604, 229.46322062727472, 189.37872271737845],
 168: [474.50432909190295, 319.6199513026192, 281.5650612571233],
 192: [609.2301332698065, 399.7143084337964, 333.28670707646245],
 336: [908.5841349053487, 728.1723528503993, 650.7431132687982],
 672: [1848.5207568587812, 1612.2188363608043, 1489.8733845259228]}

In [ ]:
policy_results = {'max_fits_1100': [1893.9663657607266, 133.45026146609104, 117.59550187337948],
 'max_fits_2100': [2875.0872626231935, 1447.6250864288265, 99.89538248542472],
 'max_fits_4200': [3430.718917641645, 2591.967671743391, 95.01342291496671],
 'max_fits_8400': [3586.4346343021443, 3006.052341175111, 93.57666588051953]}

## Select best from baseline/policy 

In [ ]:
replicas = [1, 2]
results = {"baseline": [], "policy": []}
for i in range(len(replicas)): 
    
    best_baseline = None
    for key in baseline_replica_results.keys(): 
        if best_baseline is None or baseline_replica_results[key][i] <= best_baseline: 
            best_baseline = baseline_replica_results[key][i]
    results["baseline"].append(best_baseline)
    
    best_baseline = None
    for key in lp_replica_results.keys(): 
        if best_baseline is None or lp_replica_results[key][i] <= best_baseline: 
            best_baseline = lp_replica_results[key][i]
    results["policy"].append(best_baseline)
    
results

## Plot Results 

In [ ]:
import matplotlib.pyplot as plt
import seaborn

x = 'Factor'

df = pd.DataFrame({
    x: replicas, 
    **results,
})
fig, ax1 = plt.subplots(figsize=(10, 5))
tidy = df.melt(id_vars=x).rename(columns=str.title)
seaborn.barplot(x=x, y='Value', hue='Variable', data=tidy, ax=ax1)
seaborn.despine(fig)

ax1.set(xlabel="Num Replicas", ylabel=f'MASE Loss', title='Residual Estimate Loss for Time-Series Decomposition')